In [1]:
import pandas as pd
import folium
import math
from folium.plugins import MarkerCluster
from folium import Map, FeatureGroup, Marker, LayerControl
from pandas import compat
import numpy as np
from branca.element import Template, MacroElement
compat.PY3 = True

In [2]:
#reading the file
df = pd.read_csv('new_dataset.csv')

In [3]:
df['index'] = range(0, len(df))

In [4]:
#record overlapping points
l=[]
for each in df.iterrows():
    l.append([each[1]['index'],each[1]['is_TJMaxx'],each[1]['is_Marshalls'],each[1]['is_HomeGoods'],
              each[1]['INTPTLAT'],each[1]['INTPTLON']])

df['tjover'] = [0] * len(df['INTPTLAT'])
df['marover'] =[0] * len(df['INTPTLAT'])
df['homeover'] =[0] * len(df['INTPTLAT'])

for i in df.iterrows():
    for j in l:
        if (i[1]['INTPTLAT'] == j[-2]) & (i[1]['INTPTLON']== j[-1]):
            if i[1]['is_TJMaxx'] != j[1]:
                df['tjover'][j[0]]= 1
            elif i[1]['is_Marshalls'] != j[2]:
                df['marover'][j[0]] = 1
            elif i[1]['is_HomeGoods'] != j[3]:
                df['homeover'][j[0]] = 1



<ipython-input-4-0a6e82051a0d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tjover'][j[0]]= 1
<ipython-input-4-0a6e82051a0d>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['marover'][j[0]] = 1


In [5]:
#mapping functions
def color_producer(tjmaxx, marshalls,homegoods):
    if tjmaxx == 1 :
        return 'red'
    elif marshalls == 1:
        return 'blue'
    else:
        return 'yellow' 

def store_visit(tjmaxx, marshalls,homegoods, tj_visit,m_visit,h_visit,tjover,marover,homeover):
    if tjover==1 & marover == 1:
        return str(round(tj_visit))+'/'+str(round(m_visit))+'/'+str(round(h_visit))
    elif tjmaxx == 1 :
        if marover == 1:
            return  str(round(tj_visit))+'/'+str(round(m_visit))
        elif homeover == 1:
            return  str(round(tj_visit))+'/'+str(round(h_visit))
        else:
            return str(round(tj_visit))
    elif marshalls == 1:
        if tjover == 1:
            return  str(round(tj_visit))+'/'+str(round(m_visit))
        elif homeover == 1:
            return str(round(m_visit))+'/'+str(round(h_visit))
        else:
            return str(round(m_visit))
    else:
        if tjover == 1:
            return  str(round(tj_visit))+'/'+str(round(h_visit))
        elif marover == 1:
            return str(round(m_visit))+'/'+str(round(h_visit))
        else:
            return str(round(h_visit))
    
def store_name(tjmaxx, marshalls,homegoods,tjover,marover,homeover):
    if tjover==1 & marover == 1:
        return 'TJ.Maxx/Marshalls/HomeGoods'
    elif tjmaxx == 1 :
        if marover == 1:
            return  'TJ.Maxx/Marshalls'
        elif homeover == 1:
            return  'TJ.Maxx/Homegoods'
        else:
            return 'TJ.Maxx'
    elif marshalls == 1:
        if tjover == 1:
            return  'TJ.Maxx/Marshalls'
        elif homeover == 1:
            return 'Marshalls/HomeGoods'
        else:
            return 'Marshalls'
    else:
        if tjover == 1:
            return  'TJ.Maxx/Homegoods'
        elif marover == 1:
            return 'Marshalls/HomeGoods'
        else:
            return 'HomeGoods'       


In [6]:
def drawmap_by_state(state):
    state = df[df['State']== str(state)]
    map = folium.Map(location=[state['INTPTLAT'].mean(),state['INTPTLON'].mean()], zoom_start=11)
    
    marker_cluster = folium.plugins.MarkerCluster().add_to(map)
    for lat, lng, tjmaxx, marshalls,homegoods, tj_visit,m_visit,h_visit,population,income,tjover,marover,homeover in zip(state['INTPTLAT'],state['INTPTLON'], 
                state['is_TJMaxx'],state['is_Marshalls'],state['is_HomeGoods'],
                state['TJ.Maxx'],state['Marshalls'],state['HomeGoods'],
                 state['Population'], state['MedianIncome'],
                 state['tjover'],state['marover'],state['homeover']            ):
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color_producer(tjmaxx, marshalls,homegoods),
        popup= str(store_name(tjmaxx, marshalls,homegoods,tjover,marover,homeover)) + '<br>' +
                "Visits: " + store_visit(tjmaxx, marshalls,homegoods, tj_visit,m_visit,h_visit,tjover,marover,homeover)+'<br>' 
                + "Pop: "+str(round(population)) +'<br>'+ 
                "Median Income: $"+str(round(income)) +'<br>',
        fill=True,
        fill_color=color_producer(tjmaxx, marshalls,homegoods),
        fill_opacity=0.7,
        parse_html=False).add_to(marker_cluster) 
        #map legend template
        template = """{% macro html(this, kwargs) %}

    <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>jQuery UI Draggable - Default functionality</title>
      <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

      <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
      <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

      <script>
      $( function() {
        $( "#maplegend" ).draggable({
                        start: function (event, ui) {
                            $(this).css({
                                right: "auto",
                                top: "auto",
                                bottom: "auto"
                            });
                        }
                    });
    });

      </script>
    </head>
    <body>


    <div id='maplegend' class='maplegend' 
        style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
         border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

    <div class='legend-title'>Legend (draggable!)</div>
    <div class='legend-scale'>
      <ul class='legend-labels'>
        <li><span style='background:red;opacity:0.7;'></span>T.J.Maxx</li>
        <li><span style='background:blue;opacity:0.7;'></span>Marshalls</li>
        <li><span style='background:yellow;opacity:0.7;'></span>HomeGoods</li>

      </ul>
    </div>
    </div>

    </body>
    </html>

    <style type='text/css'>
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 1px solid #999;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}"""
        macro = MacroElement()
        macro._template = Template(template)
        map.get_root().add_child(macro)
    return map

## Visualize BY STORES

### T.J.Maxx

In [7]:
tjmaxx = df[df['is_TJMaxx']==1]

In [8]:
def color_producer_state(state):
    if state == 'Florida' :
        return 'red'
    elif state == 'Tennessee':
        return 'blue'
    else:
        return 'yellow' 


# create map of Madrid using latitude and longitude values
map_tjmaxx = folium.Map(location=[tjmaxx['INTPTLAT'].mean(),tjmaxx['INTPTLON'].mean()], zoom_start=11)

marker_cluster = folium.plugins.MarkerCluster().add_to(map_tjmaxx)

# add markers to map
for lat, lng, state,visits in zip(tjmaxx['INTPTLAT'],tjmaxx['INTPTLON'], tjmaxx['State'],tjmaxx['TJ.Maxx']):

            folium.CircleMarker(
                [lat, lng],
                radius=(visits/100000),
                color=state,
                popup=str(state) + '<br>' +
                        "T.J.Maxx" + '<br>' +
                        "Visits" + str(round(visits)),
                fill=True,
                fill_color=color_producer_state(state),
                fill_opacity=0.7,
                parse_html=False).add_to(marker_cluster) 


In [9]:
#following map of next tj maxx stores in 3 states
#tjmaxx stores with states distinguished by color and marker size represents predicted total visit size
map_tjmaxx

### Marshalls

In [10]:
marshalls = df[df['is_Marshalls']==1]

In [11]:
# create map of Madrid using latitude and longitude values
map_marshalls = folium.Map(location=[marshalls['INTPTLAT'].mean(),marshalls['INTPTLON'].mean()], zoom_start=11)

marker_cluster = folium.plugins.MarkerCluster().add_to(map_marshalls)

# add markers to map
for lat, lng, state,visits in zip(marshalls['INTPTLAT'],marshalls['INTPTLON'], marshalls['State'],marshalls['Marshalls']):
    folium.CircleMarker(
        [lat, lng],
        radius=(visits/100000),
        color=state,
        popup=str(state) + '<br>' +
                "Marshalls" + '<br>' +
                "Visits" + str(round(visits)),
        fill=True,
        fill_color=color_producer_state(state),
        fill_opacity=0.7,
        parse_html=False).add_to(marker_cluster) 

In [12]:
#following map of next marshall stores in 3 states
#marshall stores with states distinguished by color and marker size represents predicted total visit size
map_marshalls

### Homegoods

In [13]:
homegoods = df[df['is_HomeGoods']==1]

In [14]:
# create map of Madrid using latitude and longitude values
map_homegoods = folium.Map(location=[homegoods['INTPTLAT'].mean(),homegoods['INTPTLON'].mean()], zoom_start=11)

marker_cluster = folium.plugins.MarkerCluster().add_to(map_homegoods)


# add markers to map
for lat, lng, state,visits in zip(homegoods['INTPTLAT'],homegoods['INTPTLON'], homegoods['State'],homegoods['Marshalls']):
    folium.CircleMarker(
        [lat, lng],
        radius=(visits/100000),
        color=state,
        popup=str(state) + '<br>' +
                "Marshalls" + '<br>' +
                "Visits" + str(round(visits)),
        fill=True,
        fill_color=color_producer_state(state),
        fill_opacity=0.7,
        parse_html=False).add_to(marker_cluster) 

In [15]:
#following map of next homegoods stores in 3 states
#homegoods stores with states distinguished by color and marker size represents predicted total visit size
map_homegoods

## Visualize BY State

### Florida 

In [16]:
drawmap_by_state('Florida')

### Alabama

In [17]:
drawmap_by_state('Alabama')

### Tennessee

In [18]:
drawmap_by_state('Tennessee')